In [ ]:
%load_ext autoreload
%autoreload 2
from modules.sftp_operations import *
import json 
import pandas_gbq
import pandas as pd
import logging
clear_logging_handlers()

#Configure loggging
logging.basicConfig(filename='SFTP_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.info('\n\n-------------New SFTP Operations Logging Instance')



with open('powerschool-420113-db919282054b.json') as json_file:
    j = json.load(json_file) 
    clever_password = j['clever_password']
    clever_username = j['clever_username']
    clever_host = j['clever_host']
    savva_password = j['savva_password']
    savva_username = j['savva_username']
    savva_host = j['savva_host']


#All BQ tables that are begin queryed are the keys. 
#How they are saved in the local dir are the values
clever_dictionary = {'Clever_schools':'schools.csv',
                    'Clever_students':'students.csv',
                    'Clever_teachers':'teachers.csv',
                    'Clever_sections':'sections.csv',
                    # 'Clever_enrollments':'enrollments.csv' #empty file, need to fix the logging
                    }

savva_dictionary = {
                    'SAVVAS_CODE_DISTRICT': 'CODE_DISTRICT.txt',
                    'SAVVAS_SCHOOL': 'SCHOOL.txt',
                    'SAVVAS_STUDENT': 'STUDENT.txt',
                    'SAVVAS_STAFF': 'STAFF.txt',
                    'SAVVAS_PIF_SECTION': 'PIF_SECTION.txt',
                    'SAVVAS_PIF_SECTION_STAFF': 'PIF_SECTION_STAFF.txt',
                    'SAVVAS_PIF_SECTION_STUDENT': 'PIF_SECTION_STUDENT.txt',
                    'SAVVAS_ENROLLMENT': 'ENROLLMENT.txt',
                    'SAVVAS_ASSIGNMENT': 'ASSIGNMENT.txt'
                }


#Instantiate Clever sftp_conn
#This needs to be done within SFTP_conn_file_exchange given the fact that Clevers uses two SFTPs
sftp_conn = SFTPConnection(
    host=clever_host,
    username=clever_username,
    password=clever_password,
    use_pool=False
)

#Import brings in Google password info from Clevers SFTP directly to powerschool_combined
#Ensure this works once populated
SFTP_conn_file_exchange(sftp_conn,
                        import_or_export = 'import',
                        sftp_folder_name='idm-sensitive-exports', 
                        local_folder_name=r'S:\SFTP\powerschool_combined',
                        use_pool=True)


#Export replicates BQ views to local dir. Follow naming convention as dictionary arg. 
SFTP_conn_file_exchange(sftp_conn,
                        import_or_export = 'export',
                        sftp_folder_name='clever_iota_file_transfer', 
                        naming_dict = clever_dictionary,
                        use_pool=False
                        )

#Export the replicated files from BQ to Clevers SFTP
# SFTP_export_dir_to_SFTP(local_dir=os.getcwd() + '\\clever_iota_file_transfer',
#                remote_dir='/',  #root dir on clevers sftp
#                sftp = sftp_conn)


# ----------------------------SAVVA piece-----------------------------------

sftp_conn = SFTPConnection(
    host=savva_host,
    username=savva_username,
    password=savva_password,
    use_pool=False
)


#Export replicates BQ views to local dir. Follow naming convention as dictionary arg. 
SFTP_conn_file_exchange(sftp_conn,
                        import_or_export = 'export',
                        sftp_folder_name='savva_iota_file_transfer', 
                        naming_dict = savva_dictionary,
                        use_pool=False
                        )

#
SFTP_export_dir_to_SFTP(local_dir=os.getcwd() + '\\savva_iota_file_transfer',
               remote_dir='/SIS',  #root dir on clevers sftp
               sftp = sftp_conn)



sftp_conn.close_all_connections()




#Current run diagnosis. 

#No files to download in folder "idm-sensitive-exports"
#Once these files are available these will send to the sftp folder powerschool_combined

#All Clever views succesfully brought down with correct naming to local dir, except Clever_enrollments
#Clever enrollments is an empty view in BQ

#Not yet ready to write these files over due to connection issues to sftp.clever.com

# ------SAVVA-------
#These are all empty views therefore unable to query and bring to local dir with naming convention
#
# SAVVAS_CODE_DISTRICT SAVVAS_SCHOOL SAVVAS_STUDENT SAVVAS_STAFF

#This file was good. Brought down from BQ, renaming succesfully, and uploaded to SAVVA SIS folder
# PIF_SECTION.txt
